In [1]:
import pandas as pd
from textblob import TextBlob
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os

In [26]:
def vader(text):
    split_para = nltk.tokenize.sent_tokenize(text)
    compound = 0
    for i in range(len(split_para)):
        polarity = analyzer.polarity_scores(split_para[i])
        compound += polarity['compound']
    return compound/len(split_para)

In [15]:
def text_blob(text):
    split_para = nltk.tokenize.sent_tokenize(text)
    comp = 0
    for i in range(len(split_para)):
        blob = TextBlob(split_para[i])
        comp += blob.sentiment.polarity
    return comp/len(split_para)

In [32]:
def get_sentiment(data):
    sentiment_list = []
    for __, row in tqdm(data.iterrows()):
        vd = vader(row['text'])
        tb = text_blob(row['text'])
        if abs(vd - tb) >= 0.15:
            if tb < 0.9:
                sentiment_list.append(tb)
            else:
                sentiment_list.append(vd)
        else:
            sentiment_list.append(vd)
    return sentiment_list

In [33]:
def scale_output(senti_list):
    for i in range(len(senti_list)):
        if senti_list[i] < -0.6:
            senti_list[i] = 1
        elif senti_list[i] >= -0.6 and senti_list[i] < -0.2:
            senti_list[i] = 2
        elif senti_list[i] >= -0.2 and senti_list[i] < 0.2:
            senti_list[i] = 3
        elif senti_list[i] >= 0.2 and senti_list[i] < 0.6:
            senti_list[i] = 4
        else:
            senti_list[i] = 5
    return senti_list

In [31]:
from tqdm import tqdm

In [35]:
analyzer = SentimentIntensityAnalyzer()
filelist = os.listdir('/Users/paurax/Downloads/dv_proj/review/')
for file in filelist:
    data = pd.read_csv('/Users/paurax/Downloads/dv_proj/review/' + file)
    sentiment_list = get_sentiment(data)
    sentiment_list = scale_output(sentiment_list)
    data['sentiment'] = sentiment_list
    dest_path = '/Users/paurax/Downloads/dv_proj/output/' + file 
    data.to_csv(dest_path)

1041803it [34:32, 502.70it/s] 
